In [ ]:

!pip install imbalanced-learn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings("ignore")

In [ ]:

train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")

print(train.head())
print(test.head())

In [ ]:
# ------------------------------
# 🔍 MISSING VALUE REPORT
# ------------------------------
print("Missing values in TRAIN:\n", train.isnull().sum())
print("\nMissing values in TEST:\n", test.isnull().sum())

# ------------------------------
# 🔧 IMPUTE MISSING VALUES
# ------------------------------

# Numeric columns -> median
num_cols = train.select_dtypes(include=['number']).columns

for col in num_cols:
    median_val = train[col].median()
    train[col].fillna(median_val, inplace=True)
    if col in test:
        test[col].fillna(median_val, inplace=True)

# Categorical columns -> mode
cat_cols = train.select_dtypes(include=['object']).columns

for col in cat_cols:
    mode_val = train[col].mode()[0]
    train[col].fillna(mode_val, inplace=True)
    if col in test:
        test[col].fillna(mode_val, inplace=True)

print("\nAll missing values handled successfully.")


In [ ]:

target_col = list(set(train.columns) - set(test.columns))

if len(target_col) != 1:
    raise ValueError("ERROR: Could not uniquely detect target column.")

target_col = target_col[0]
print("Detected target column:", target_col)

In [ ]:

# Correlation Heatmap
plt.figure(figsize=(12,6))
sns.heatmap(train.corr(), cmap='coolwarm')
plt.title("Correlation Heatmap")
plt.show()

# Boxplots for numeric columns
num_cols = train.select_dtypes(include=np.number).columns

for col in num_cols:
    plt.figure(figsize=(6,3))
    sns.boxplot(x=train[col])
    plt.title(f"Boxplot of {col}")
    plt.show()

In [ ]:

def remove_outliers(df, cols):
    df_clean = df.copy()
    for col in cols:
        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
        df_clean = df_clean[(df_clean[col] >= lower) & (df_clean[col] <= upper)]
    return df_clean

train_clean = remove_outliers(train, num_cols)
print("Original:", train.shape, "| Cleaned:", train_clean.shape)

In [ ]:
label_encoders = {}

X = train_clean.drop(columns=[target_col])
y = train_clean[target_col]

# Encode categorical columns safely
for col in X.columns:
    if X[col].dtype == "object":
        le = LabelEncoder()
        X[col] = le.fit_transform(X[col].astype(str))

        # Safe transform for test-set unseen labels
        test[col] = test[col].astype(str).apply(lambda x:
                    x if x in le.classes_ else "unknown")

        le.classes_ = np.append(le.classes_, "unknown")
        test[col] = le.transform(test[col])

        label_encoders[col] = le

# Encode target if needed
target_encoder = None
if y.dtype == "object":
    target_encoder = LabelEncoder()
    y = target_encoder.fit_transform(y)


In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)

sm = SMOTE(random_state=42)
X_train, y_train = sm.fit_resample(X_train, y_train)


In [ ]:
param_dist = {
    "n_estimators": [100, 200, 300, 500],
    "max_depth": [None, 10, 20, 30],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "bootstrap": [True, False]
}

rf = RandomForestClassifier(random_state=42)

random_search = RandomizedSearchCV(
    rf, param_distributions=param_dist,
    n_iter=20, scoring="roc_auc", cv=3, n_jobs=-1, verbose=1
)

random_search.fit(X_train, y_train)
best_model = random_search.best_estimator_
print("Best Params:", random_search.best_params_)


In [ ]:
y_pred = best_model.predict(X_valid)
y_proba = best_model.predict_proba(X_valid)[:,1]

print(classification_report(y_valid, y_pred))
print("AUC Score:", roc_auc_score(y_valid, y_proba))

# ROC Curve
fpr, tpr, _ = roc_curve(y_valid, y_proba)
plt.plot(fpr, tpr)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.show()


In [ ]:
# Preprocess test
test_processed = test.copy()

for col in test_processed.columns:
    if col in label_encoders:
        test_processed[col] = label_encoders[col].transform(test_processed[col].astype(str))

test_scaled = scaler.transform(test_processed)

# Predict
test_pred = best_model.predict(test_scaled)

# Inverse target encoding if needed
if target_encoder is not None:
    test_pred = target_encoder.inverse_transform(test_pred)

# Build submission
id_col = test.columns[0]  # first column = id
submission = pd.DataFrame({
    "id": test[id_col],
    "target": test_pred
})

submission.to_csv("submission.csv", index=False)
submission.head()
